In [ ]:
%pip install xgboost --quiet

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
import pickle

data = pd.read_csv("posture_new_data.csv")

data = data.sample(frac=1, random_state=42).reset_index(drop=True)

print(data.head())
print(data['label'].value_counts())

data = data[~((data[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].abs() < 0.05).all(axis=1))]

print("After cleaning:", data.shape)
print(data['label'].value_counts())

le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])  # bad=0, good=1

X = data[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values
y = data['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='logloss',
    random_state=42
)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

print("\n✅ XGBoost Model Evaluation")
print("Accuracy:", round(accuracy_score(y_test, y_pred), 4))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))

with open("xgb_new_posture.pkl", "wb") as f:
    pickle.dump(xgb, f)

with open("scaler_xgb_posture.pkl", "wb") as f:
    pickle.dump(scaler, f)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
     ax    ay    az     gx    gy    gz label
0 -0.24  0.94 -0.27   7.00  6.51  1.34  good
1 -0.22  0.97 -0.18  -9.24 -9.69 -2.21   bad
2 -0.27  0.92  0.03   3.02  0.75  0.05   bad
3 -0.23  0.99 -0.31  10.78  3.83  2.73   bad
4 -0.27  1.00 -0.13  -7.67  0.24 -1.66   bad
label
good    1129
bad     1000
Name: count, dtype: int64
After cleaning: (2129, 7)
label
good    1129
bad     1000
Name: count, dtype: int64

✅ XGBoost Model Evaluation
Accuracy: 0.9225

Confusion Matrix:
 [[182  18]
 [ 15 211]]

Classification Report:
               precision    recall  f1-score   support

         bad       0.92      0.91      0.92       200
        good       0.92      0.93      0.93       226

    accuracy                           0.92       426
   macro avg       0.92      0.92      0.92       426
weighted avg       0.92      0.92      0.92       426



In [2]:

import pandas as pd
df = pd.read_csv("posture_data_new_good.csv")
print(df.describe())

                ax           ay           az           gx           gy  \
count  1129.000000  1129.000000  1129.000000  1129.000000  1129.000000   
mean     -0.240328     0.937706    -0.214322     1.491878     1.423136   
std       0.049266     0.166287     0.072505     5.081742     3.789672   
min      -0.330000    -0.000000    -0.410000   -28.920000   -18.550000   
25%      -0.260000     0.960000    -0.260000    -0.480000    -0.340000   
50%      -0.250000     0.970000    -0.220000     1.700000     1.750000   
75%      -0.230000     0.980000    -0.170000     3.940000     3.370000   
max      -0.000000     1.050000    -0.000000    30.740000    19.850000   

                gz  
count  1129.000000  
mean      0.227927  
std       1.889758  
min      -6.950000  
25%      -0.680000  
50%       0.370000  
75%       1.240000  
max       7.500000  


In [3]:

import pandas as pd
df1 = pd.read_csv("posture_data_new_bad.csv")
print(df1.describe())

               ax           ay           az           gx           gy  \
count  1000.00000  1000.000000  1000.000000  1000.000000  1000.000000   
mean     -0.19875     0.953970    -0.266410     0.203570    -0.029600   
std       0.07275     0.042635     0.210372    10.869927     5.170504   
min      -0.32000     0.780000    -0.780000   -37.340000   -29.740000   
25%      -0.25000     0.930000    -0.402500    -7.682500    -3.010000   
50%      -0.22000     0.960000    -0.220000     0.380000    -0.100000   
75%      -0.17000     0.980000    -0.100000     7.400000     3.182500   
max       0.05000     1.080000     0.090000    29.850000    17.900000   

                gz  
count  1000.000000  
mean     -0.254370  
std       2.823108  
min     -18.650000  
25%      -1.955000  
50%      -0.265000  
75%       1.605000  
max      10.550000  
